In [1]:
SANDBOX_NAME = # Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/data/"



# Spark ML Problema de Regresión

En este notebook abordaremos el problema de Machine Learning Supervisado de regresión. Trabajaremos con el dataset Boston Housing que contiene información sobre las diferentes características de casas en la ciudad de Boston. Utilizaremos como variable objetivo el precio de las casas. Accederemos a este a través de la librería de ML de Python scikit-learn. Ajustaremos distintos modelos y compararemos los resultados obtenidos en éstos.



### Crear SparkSession
Nota: en Datio no es necesario crear la sesión de Spark ya al iniciar un notebook con el kernel PySpark Python3 - Spark 2.1.0 se crea automáticamente.

In [2]:
# Respuesta
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()



### Cargamos los datos en un DataFrame de Spark
Cargamos los datos de scikit-learn y los consolidamos en un DataFrame de Spark.

In [3]:
# Respuesta

import pandas as pd
from sklearn.datasets import load_boston

boston_dataset = load_boston()
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
# We add the target variable to our Pandas DataFrame
boston['MEDV'] = boston_dataset.target
# We create a Spark DataFrame from the Pandas DataFrame
boston = spark.createDataFrame(boston)
boston.show()

+-------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|
+-------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+
|0.00632|18.0| 2.31| 0.0|0.538|6.575| 65.2|  4.09|1.0|296.0|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07| 0.0|0.469|6.421| 78.9|4.9671|2.0|242.0|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07| 0.0|0.469|7.185| 61.1|4.9671|2.0|242.0|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18| 0.0|0.458|6.998| 45.8|6.0622|3.0|222.0|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18| 0.0|0.458|7.147| 54.2|6.0622|3.0|222.0|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18| 0.0|0.458| 6.43| 58.7|6.0622|3.0|222.0|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87| 0.0|0.524|6.012| 66.6|5.5605|5.0|311.0|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87| 0.0|0.524|6.172| 96.1|5.9505|5.0|311.0|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87| 0.0|0.524|5.631|100.0|



Vemos el schema.

In [4]:
# Respuesta
boston.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: double (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: double (nullable = true)
 |-- TAX: double (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MEDV: double (nullable = true)





Podemos ver la descripción de las variables del objeto cargado de scikit-learn.

In [5]:
# Respuesta

print(boston_dataset.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu



### Pasos previos

#### Vector Assembler
Para ajustar un modelo en Spark necesitamos indicar qué variables se van a utilizar como variables independientes. A través del parámetro featuresCol de los distintos algoritmos, se le indica la variable que contiene la salida del  VectorAssembler con las variables independientes. 

Hacemos el VectorAssembler con todas las variables con excepcion del target.

In [6]:
# Respuesta
from pyspark.ml.feature import VectorAssembler

variables_vector_assembler = list(set(boston.columns) - set(['MEDV'])) # We do not add the target variable
vectorassembler = VectorAssembler(inputCols = variables_vector_assembler, outputCol = 'assembled_features')
boston = vectorassembler.transform(boston)
boston.show()

+-------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+--------------------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|MEDV|  assembled_features|
+-------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31| 0.0|0.538|6.575| 65.2|  4.09|1.0|296.0|   15.3| 396.9| 4.98|24.0|[0.0,4.98,65.2,1....|
|0.02731| 0.0| 7.07| 0.0|0.469|6.421| 78.9|4.9671|2.0|242.0|   17.8| 396.9| 9.14|21.6|[0.0,9.14,78.9,2....|
|0.02729| 0.0| 7.07| 0.0|0.469|7.185| 61.1|4.9671|2.0|242.0|   17.8|392.83| 4.03|34.7|[0.0,4.03,61.1,2....|
|0.03237| 0.0| 2.18| 0.0|0.458|6.998| 45.8|6.0622|3.0|222.0|   18.7|394.63| 2.94|33.4|[0.0,2.94,45.8,3....|
|0.06905| 0.0| 2.18| 0.0|0.458|7.147| 54.2|6.0622|3.0|222.0|   18.7| 396.9| 5.33|36.2|[0.0,5.33,54.2,3....|
|0.02985| 0.0| 2.18| 0.0|0.458| 6.43| 58.7|6.0622|3.0|222.0|   18.7|394.12| 5.21|28.7|[0.0,5.21,58.7,3....|
|0.08829|12.5| 7.87| 0.0|0.5



#### División train/test
Realizamos la division train/test (o train/validation) para medir el desempeño del modelo tras el ajuste.

In [7]:
# Respuesta

boston_train, boston_test = boston.randomSplit([0.8,0.2])



### Regresión Lineal
Modelo matemático usado para aproximar la relación de dependencia entre una variable dependiente $Y$, las variables independientes $X_i$ y un término aleatorio $ε$. Se expresa a traves de la siguiente ecuación:

$$Y = \beta_0+\beta_1*X_1+\beta_2*X_2+...+\beta_p*X_p+ε$$

donde 
- $Y$ es la variable dependiente, explicada o target,
- $X_i$ son las variables explicativas, independientes o regresoras,
- $\beta_i$ son los parametros que miden la influencia que tienen las variables explicativas sobre el target,

para todo $0<=i<=p$.

Ajustamos un modelo de regresión y sacamos los valores reales y los predichos.

In [8]:
# Respuesta
from pyspark.ml.regression import LinearRegression

linear_regression = LinearRegression(featuresCol='assembled_features', labelCol='MEDV')
linear_regression_model = linear_regression.fit(boston_train)
boston_test_linear_regression = linear_regression_model.transform(boston_test)
boston_test_linear_regression.select(['MEDV','prediction']).show()

+----+------------------+
|MEDV|        prediction|
+----+------------------+
|31.6| 32.67371908781077|
|34.9| 33.38533918735798|
|22.9| 24.97504143737227|
|23.9|27.088272888403544|
|22.6| 27.15961849422017|
|25.0| 27.64644312692204|
|21.2|21.788123942836137|
|23.6|30.903239789194487|
|24.1| 25.69443240323216|
|22.9|23.456236064211744|
|19.7| 21.39743320819953|
|21.7|20.599737094317067|
|28.4|28.640945117479966|
|18.8|21.319644735065754|
|26.6|27.885786896806316|
|20.9|20.891588086501976|
|19.5|18.765585748709753|
|21.7| 21.84251616338134|
|24.7| 24.50571357817384|
|18.3|20.883807380689294|
+----+------------------+
only showing top 20 rows





### Arbol de Decisión
Modelo predictivo que se construye ejecutando una partición binaria recursiva de los datos identificando las variables y los puntos de corte de las mismas que mejor determinan el valor de la variable objetivo. En el entrenamiento son importantes los parámetros: medida de impureza y criterio de parada (normalmente profundidad).

Ajustamos un árbol de decisión para nuestro conjunto de datos y sacamos los valores reales y los predichos.

In [9]:
# Respuesta

from pyspark.ml.regression import DecisionTreeRegressor

decision_tree_regression = DecisionTreeRegressor(featuresCol='assembled_features', labelCol='MEDV')
decision_tree_regression_model = decision_tree_regression.fit(boston_train)
boston_test_decision_tree_regression = decision_tree_regression_model.transform(boston_test)
boston_test_decision_tree_regression.select(['MEDV','prediction']).show()

+----+------------------+
|MEDV|        prediction|
+----+------------------+
|31.6|  32.8095238095238|
|34.9|  32.8095238095238|
|22.9|         23.978125|
|23.9| 27.39259259259259|
|22.6|21.074789915966388|
|25.0| 27.39259259259259|
|21.2|21.074789915966388|
|23.6|  32.8095238095238|
|24.1|         23.978125|
|22.9|21.074789915966388|
|19.7|21.074789915966388|
|21.7|21.074789915966388|
|28.4| 27.39259259259259|
|18.8|20.699999999999996|
|26.6| 27.39259259259259|
|20.9|21.074789915966388|
|19.5|16.117073170731707|
|21.7|21.074789915966388|
|24.7|         23.978125|
|18.3|20.699999999999996|
+----+------------------+
only showing top 20 rows





### Random Forest
Modelo predictivo basado en árboles de decisión. Construye varios árboles tomando muestras del conjunto de datos (boosting) y muestras del conjunto de variables. Realiza la predicción para cada nuevo registro pasandolo por cada uno de los árboles y promediando los resultados obtenidos.

Ajustamos un random forest y obtenemos los valores reales y los predichos. 

In [10]:
# Respuesta
from pyspark.ml.regression import RandomForestRegressor

random_forest = RandomForestRegressor(featuresCol='assembled_features', labelCol='MEDV')
random_forest_model = random_forest.fit(boston_train)
boston_test_random_forest = random_forest_model.transform(boston_test)
boston_test_random_forest.select(['MEDV','prediction']).show()

+----+------------------+
|MEDV|        prediction|
+----+------------------+
|31.6|33.042758209794464|
|34.9| 37.49463427817052|
|22.9|23.806929683515303|
|23.9|25.067190830049377|
|22.6| 24.44257786642228|
|25.0| 26.73127264710039|
|21.2|21.045540543591056|
|23.6|31.873814731121154|
|24.1|23.896402042797657|
|22.9|20.822018953254847|
|19.7|21.019563176280172|
|21.7|20.057134466137548|
|28.4|26.938687963918262|
|18.8|19.945213933815605|
|26.6|30.074980333582552|
|20.9|21.611580564337153|
|19.5| 17.93289634889239|
|21.7|20.959670489576055|
|24.7|24.013625085814155|
|18.3|20.141509045873374|
+----+------------------+
only showing top 20 rows



 

# Evaluación de modelos



Para comparar el desempeño de los modelos ajustados y poder seleccionar el mejor de ellos disponemos de diversas métricas. Algunas de ellas son:
- Error cuadrático medio
- Raíz del error cuadrático medio
- R cuadrado
- Error absoluto medio

Como ejemplo obtenemos el RMSE y el MAE de los modelos ajustados.

In [11]:
# Respuesta

from pyspark.ml.evaluation import RegressionEvaluator

""" Possibilities::
metric name in evaluation - one of:
                       rmse - root mean squared error (default)
                       mse - mean squared error
                       r2 - r^2 metric
                       mae - mean absolute error.
"""

rmse = RegressionEvaluator(predictionCol='prediction', labelCol='MEDV', metricName='rmse')
mae = RegressionEvaluator(predictionCol='prediction', labelCol='MEDV', metricName='mae')



Imprimir métricas para los distintos modelos

In [12]:
# Respuesta

print("RMSE for a linear regression: {}".format(rmse.evaluate(boston_test_linear_regression)))
print("RMSE for a decision tree: {}".format(rmse.evaluate(boston_test_decision_tree_regression)))
print("RMSE for a random forest: {}\n".format(rmse.evaluate(boston_test_random_forest)))

print("MAE for a regresión lineal: {}".format(mae.evaluate(boston_test_linear_regression)))
print("MAE for a decision tree: {}".format(mae.evaluate(boston_test_decision_tree_regression)))
print("MAE for a random forest: {}".format(mae.evaluate(boston_test_random_forest)))

RMSE for a linear regression: 4.056855814284501
RMSE for a decision tree: 3.410485523217875
RMSE for a random forest: 2.798164095399894

MAE for a regresión lineal: 3.054713938595181
MAE for a decision tree: 2.409132195678887
MAE for a random forest: 2.0325956959757283




Observando los valores obtenidos en las métricas de evaluación de modelos decidiríamos quedarnos con el random forest.